In [12]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.converters import QubitConverter

# Step 1: 定义分子（H2，sto-3g，bond length=0.74Å）
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.74", basis="sto3g")
es_problem = ElectronicStructureProblem(driver)

# Step 2: 提取电子结构问题（1-electron integrals, etc）
second_q_op = es_problem.second_q_ops()[0]

# Step 3: 映射为量子比特哈密顿量（使用 Jordan-Wigner）
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(second_q_op)

# 输出 Qubit Hamiltonian（你可以用 qubit_op.to_matrix() 得到 numpy 矩阵）
print(qubit_op)


ModuleNotFoundError: No module named 'qiskit_nature.second_q.converters'

In [2]:
from openfermionpyscf import run_pyscf
from openfermion import MolecularData
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_sparse_operator

# Step 1: 定义分子信息
geometry = [("H", (0, 0, 0)), ("H", (0, 0, 0.74))]
basis = "sto-3g"
multiplicity = 1
charge = 0

# Step 2: 创建 MolecularData 对象
molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

# Step 3: 用 PySCF 进行计算
molecule = run_pyscf(molecule, run_scf=True, run_fci=True)

# Step 4: 转换为 qubit 哈密顿量
fermion_hamiltonian = molecule.get_molecular_hamiltonian()
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)

# Step 5: 变为稀疏矩阵（numpy 格式）
H_c = get_sparse_operator(qubit_hamiltonian).toarray()

# 可选：打印结果
import numpy as np
np.set_printoptions(precision=4, suppress=True)
print(H_c)


[[ 0.7151+0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.24  +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.24  +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j  0.4626+0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.1812+0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j -0.5382+0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j
   0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.

In [1]:
import numpy as np
print(np.__version__)


1.26.4


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# 自动从 H_c 推出 qubit 数（H_c 已由 openfermion 生成）
n_qubits = int(np.log2(H_c.shape[0]))

# Step 2: Driver Hamiltonian（H_d = sum X_i）
X = np.array([[0, 1], [1, 0]])
I = np.eye(2)

def build_driver_hamiltonian(n):
    H_d = np.zeros((2**n, 2**n), dtype=complex)
    for i in range(n):
        ops = [I] * n
        ops[i] = X
        H_d += kron_n(*ops)
    return H_d

def kron_n(*args):
    result = args[0]
    for op in args[1:]:
        result = np.kron(result, op)
    return result

H_d = build_driver_hamiltonian(n_qubits)

# Step 3: commutator Hamiltonian
comm_h = 1j * (H_d @ H_c - H_c @ H_d)

# Step 4: 初始化量子态 |+>^n
qc = QuantumCircuit(n_qubits)
for i in range(n_qubits):
    qc.h(i)
psi = Statevector.from_instruction(qc).data

# Step 5: 单步演化
def single_falqon_unitary(beta_k, delta_t, H_cost, H_driver):
    U_c = expm(-1j * H_cost * delta_t)
    U_d = expm(-1j * H_driver * beta_k * delta_t)
    return U_d @ U_c

# Step 6: 期望值
def measure_expectation(state, H):
    return np.real(np.vdot(state, H @ state))

# Step 7: 主循环
def run_fbqa_state_tracking(H, H_d, comm_h, n_steps, beta_1, delta_t):
    beta_list = [beta_1]
    energy_list = []
    state = psi
    for i in range(n_steps):
        exp_comm = measure_expectation(state, comm_h)
        next_beta = -1  * exp_comm
        beta_list.append(next_beta)
        U_k = single_falqon_unitary(next_beta, delta_t, H, H_d)
        state = U_k @ state
        energy = measure_expectation(state, H)
        energy_list.append(energy)
        if i % 100 == 0 or i == n_steps - 1:
            print(f"Step {i+1:4d}: E = {energy:.8f} Ha, β = {next_beta:.6f}")
    return beta_list, energy_list

# Step 8: 运行
n_steps = 2000
delta_t = 0.1
beta_1 = 0.0

res_beta, res_energies = run_fbqa_state_tracking(H_c, H_d, comm_h, n_steps, beta_1, delta_t)

# Step 9: 可视化
plt.plot(range(1, n_steps + 1), res_energies)
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartree)")
plt.title("FBQA with state tracking (Qiskit + OpenFermion, Hydrogen molecule)")
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'H_c' is not defined

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.converters import QubitConverter

from qiskit.opflow import PauliSumOp

# Step 1: 设置分子（氢分子，sto-3g，0.74Å）
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.74", basis="sto3g")
problem = ElectronicStructureProblem(driver)

# Step 2: 使用活性空间压缩为 2-qubit（2个轨道、2个电子）
transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=2)
problem = transformer.transform(problem)

# Step 3: Jordan-Wigner 映射 + 自动压缩
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

qubit_op = converter.convert(problem.second_q_ops()[0], num_particles=problem.num_particles)

# Step 4: 转换为 numpy 矩阵 H_c
H_c = qubit_op.to_matrix()

# Step 5: 输出系数表达式
print("Pauli Hamiltonian:")
print(qubit_op)

import numpy as np
np.set_printoptions(precision=4, suppress=True)
print("H_c matrix:\n", H_c)



ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [6]:
import qiskit_nature
print(qiskit_nature.__version__)


0.6.2


In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper


ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/root/miniconda3/envs/qiskit_clean/lib/python3.9/site-packages/qiskit/primitives/__init__.py)